# Step5: Post-Processing function step 2

In [6]:
import os, sys, time, importlib
import osmnx

import geopandas as gpd
import pandas as pd
import networkx as nx
import numpy as np
sys.path.append("../../../GOSTnets")
import GOSTnets as gn

from shapely.geometry import LineString, Point

In [7]:
# This is a Jupyter Notebook extension which reloads all of the modules whenever you run the code
# This is optional but good if you are modifying and testing source code
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
from GOSTnets.load_traffic2 import *

In [9]:
# read graph
#G = nx.read_gpickle('./sri_lanka_unclean2_w_time_largest_20200616.pickle')

In [10]:
#len(G.edges)

In [11]:
# import shapefile as GPD
study_area = gpd.read_file("./airport_output_edges_20200628_traffic_mean_merged/airport_weighted_sec_saved_edges_all_merged_traffic_mean.shp")

In [12]:
study_area

,o,d,w,sec_saved,imp_cost,mean_speed,length,infra_type,weighted_s,max_speed,min_speed,mode,osm_id,speed,time,geometry
0,31363728,2423962289,1,0.0,0.0,24.037202,0.047391,trunk,0.0,35.0,6.0,drive,303764438.0,24.037202,7.097606,"LINESTRING (80.23915 6.03267, 80.23884 6.03296)"
1,31363729,5436876436,1,0.0,0.0,24.477679,0.033225,trunk,0.0,39.0,5.0,drive,303764438.0,24.477679,4.886527,"LINESTRING (80.23701 6.03364, 80.23671 6.03371)"
2,31363730,5436861551,1,0.0,0.0,27.100694,0.019258,trunk,0.0,38.0,5.0,drive,303764438.0,27.100694,2.558145,"LINESTRING (80.23550 6.03401, 80.23534 6.03408)"
3,31363731,3776002089,1,0.0,0.0,33.566964,0.024058,trunk,0.0,47.0,6.0,drive,303764438.0,33.566964,2.580217,"LINESTRING (80.23353 6.03495, 80.23332 6.03501)"
4,31363732,3776002111,1,0.0,0.0,32.070437,0.033973,trunk,0.0,47.0,9.0,drive,303764438.0,32.070437,3.813516,"LINESTRING (80.23266 6.03513, 80.23236 6.03521)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18984,7291030425,637812493,1,0.0,0.0,49.471726,0.050748,primary,0.0,63.0,38.0,drive,780854939.0,49.471726,3.692885,"LINESTRING (79.90117 7.09067, 79.90149 7.09100)"
18985,7328966439,1835003964,1,0.0,0.0,NaN,0.005110,trunk,0.0,NaN,NaN,drive,381745790.0,40.000000,0.459934,"LINESTRING (80.62800 7.11547, 80.62797 7.11543)"
18986,7328980475,1835004762,1,0.0,0.0,NaN,0.004123,trunk,0.0,NaN,NaN,drive,381745790.0,40.000000,0.371046,"LINESTRING (80.62621 7.11569, 80.62618 7.11569)"
18987,7354478607,7354478608,1,0.0,0.0,NaN,0.033878,trunk,0.0,NaN,NaN,drive,205583333.0,40.000000,3.048984,"LINESTRING (79.84758 7.21358, 79.84771 7.21330)"


In [13]:
sorted_study_area = study_area.sort_values(by=['weighted_s'], ascending=False)

In [14]:
sorted_study_area[:15]

,o,d,w,sec_saved,imp_cost,mean_speed,length,infra_type,weighted_s,max_speed,min_speed,mode,osm_id,speed,time,geometry
5720,3450071446,2093606557,1,114.442632,5354.826351,29.000000,2.194998,secondary,114.442632,29.0,29.0,drive,199357860.0,29.000000,272.482456,"MULTILINESTRING ((80.61201 7.55608, 80.61177 7..."
5787,5647606451,4982371638,1,107.780874,1446.278072,39.285160,5.488517,secondary,107.780874,42.0,2.0,drive,371237411.0,39.285218,502.954126,"MULTILINESTRING ((80.55331 7.57692, 80.55324 7..."
3252,1235783744,1427372407,2,50.727777,18855.520045,30.272022,1.085464,secondary,101.455553,40.0,28.0,drive,107537454.0,30.319940,128.881157,"MULTILINESTRING ((80.17251 7.25353, 80.17330 7..."
2256,3800305939,1847947581,5,19.218641,4900.098864,26.354138,0.301721,secondary,96.093204,37.0,5.0,drive,48652919.0,26.529762,40.942578,"MULTILINESTRING ((79.94844 7.16975, 79.94856 7..."
5718,2093606556,3755273539,1,93.076372,19533.582989,23.000000,1.101212,secondary,93.076372,23.0,23.0,drive,199358003.0,23.000000,172.363653,"MULTILINESTRING ((80.62187 7.54740, 80.62184 7..."
5717,3755273540,2093606469,1,87.287466,1760.894454,24.667450,1.106191,secondary,87.287466,17.0,16.0,drive,199358003.0,16.995040,166.933184,"MULTILINESTRING ((80.62102 7.53839, 80.62106 7..."
3535,6546000232,1427372101,2,42.605424,819.640203,36.000000,1.521622,secondary,85.210849,36.0,36.0,drive,107537454.0,36.000000,152.162230,"MULTILINESTRING ((80.18625 7.24612, 80.18626 7..."
3516,1427372055,617967898,2,35.982691,5186.113230,30.896514,0.809416,secondary,71.965382,31.0,24.0,drive,107537454.0,30.913194,94.260642,"MULTILINESTRING ((80.19585 7.23294, 80.19604 7..."
11801,3804860939,617823368,2,27.390833,15599.413502,9.000000,0.083509,secondary,54.781666,9.0,9.0,drive,352065920.0,9.000000,33.403455,"LINESTRING (80.12829 7.24751, 80.12905 7.24756)"
7640,3069814342,3450046074,1,52.834367,5239.061357,28.658097,1.013358,secondary,52.834367,29.0,29.0,drive,503548865.0,29.000000,125.796112,"MULTILINESTRING ((80.55149 7.56911, 80.55156 7..."


In [15]:
total_cost = 0
total_count = 0

In [16]:
def sort_weighted_s(sec_saved, imp_cost):
    global total_cost
    global total_count
    if total_cost < 20000000:
        print(f"total_cost before:{total_cost}")
        print(f"imp_cost:{imp_cost}")
        total_cost = total_cost + imp_cost
        total_count += 1
        print(f"sec_saved:{sec_saved}")
        print(f"total_cost:{total_cost}")
        print(f"total_count:{total_count}")


In [17]:
#sorted_study_area['weighted_s'] = 
sorted_study_area.apply(lambda x: sort_weighted_s(x['sec_saved'],x['imp_cost']),axis=1)

total_cost before:0
imp_cost:5354.826350534842
sec_saved:114.44263158346023
total_cost:5354.826350534842
total_count:1
total_cost before:5354.826350534842
imp_cost:1446.2780723870906
sec_saved:107.78087377545062
total_cost:6801.104422921933
total_count:2
total_cost before:6801.104422921933
imp_cost:18855.52004493163
sec_saved:50.72777666843816
total_cost:25656.624467853562
total_count:3
total_cost before:25656.624467853562
imp_cost:4900.098864121054
sec_saved:19.218640879714368
total_cost:30556.723331974616
total_count:4
total_cost before:30556.723331974616
imp_cost:19533.58298856508
sec_saved:93.07637241623117
total_cost:50090.30632053969
total_count:5
total_cost before:50090.30632053969
imp_cost:1760.8944535295236
sec_saved:87.28746582421992
total_cost:51851.20077406921
total_count:6
total_cost before:51851.20077406921
imp_cost:819.6402032523664
sec_saved:42.60542446042511
total_cost:52670.84097732158
total_count:7
total_cost before:52670.84097732158
imp_cost:5186.113229996755
sec_sa

total_cost before:4338109.566229328
imp_cost:4752.599649444527
sec_saved:0.216293119458687
total_cost:4342862.165878772
total_count:377
total_cost before:4342862.165878772
imp_cost:1000.5739729678543
sec_saved:0.645905367052642
total_cost:4343862.739851739
total_count:378
total_cost before:4343862.739851739
imp_cost:8797.829786539824
sec_saved:0.633077838574791
total_cost:4352660.569638279
total_count:379
total_cost before:4352660.569638279
imp_cost:10900.81217531194
sec_saved:0.314548726588259
total_cost:4363561.381813591
total_count:380
total_cost before:4363561.381813591
imp_cost:7208.652173810127
sec_saved:0.204043542995731
total_cost:4370770.033987401
total_count:381
total_cost before:4370770.033987401
imp_cost:8309.03424029791
sec_saved:0.610023927094371
total_cost:4379079.068227699
total_count:382
total_cost before:4379079.068227699
imp_cost:26470.928834485352
sec_saved:0.609606462601633
total_cost:4405549.997062184
total_count:383
total_cost before:4405549.997062184
imp_cost:11

total_cost before:4977348.730753487
imp_cost:0.0
sec_saved:0.0
total_cost:4977348.730753487
total_count:577
total_cost before:4977348.730753487
imp_cost:0.0
sec_saved:0.0
total_cost:4977348.730753487
total_count:578
total_cost before:4977348.730753487
imp_cost:0.0
sec_saved:0.0
total_cost:4977348.730753487
total_count:579
total_cost before:4977348.730753487
imp_cost:0.0
sec_saved:0.0
total_cost:4977348.730753487
total_count:580
total_cost before:4977348.730753487
imp_cost:0.0
sec_saved:0.0
total_cost:4977348.730753487
total_count:581
total_cost before:4977348.730753487
imp_cost:0.0
sec_saved:0.0
total_cost:4977348.730753487
total_count:582
total_cost before:4977348.730753487
imp_cost:0.0
sec_saved:0.0
total_cost:4977348.730753487
total_count:583
total_cost before:4977348.730753487
imp_cost:0.0
sec_saved:0.0
total_cost:4977348.730753487
total_count:584
total_cost before:4977348.730753487
imp_cost:0.0
sec_saved:0.0
total_cost:4977348.730753487
total_count:585
total_cost before:4977348.73

total_count:876
total_cost before:4977348.730753487
imp_cost:0.0
sec_saved:0.0
total_cost:4977348.730753487
total_count:877
total_cost before:4977348.730753487
imp_cost:0.0
sec_saved:0.0
total_cost:4977348.730753487
total_count:878
total_cost before:4977348.730753487
imp_cost:0.0
sec_saved:0.0
total_cost:4977348.730753487
total_count:879
total_cost before:4977348.730753487
imp_cost:0.0
sec_saved:0.0
total_cost:4977348.730753487
total_count:880
total_cost before:4977348.730753487
imp_cost:0.0
sec_saved:0.0
total_cost:4977348.730753487
total_count:881
total_cost before:4977348.730753487
imp_cost:0.0
sec_saved:0.0
total_cost:4977348.730753487
total_count:882
total_cost before:4977348.730753487
imp_cost:0.0
sec_saved:0.0
total_cost:4977348.730753487
total_count:883
total_cost before:4977348.730753487
imp_cost:0.0
sec_saved:0.0
total_cost:4977348.730753487
total_count:884
total_cost before:4977348.730753487
imp_cost:0.0
sec_saved:0.0
total_cost:4977348.730753487
total_count:885
total_cost b

total_cost before:4977348.730753487
imp_cost:0.0
sec_saved:0.0
total_cost:4977348.730753487
total_count:1218
total_cost before:4977348.730753487
imp_cost:0.0
sec_saved:0.0
total_cost:4977348.730753487
total_count:1219
total_cost before:4977348.730753487
imp_cost:0.0
sec_saved:0.0
total_cost:4977348.730753487
total_count:1220
total_cost before:4977348.730753487
imp_cost:0.0
sec_saved:0.0
total_cost:4977348.730753487
total_count:1221
total_cost before:4977348.730753487
imp_cost:0.0
sec_saved:0.0
total_cost:4977348.730753487
total_count:1222
total_cost before:4977348.730753487
imp_cost:0.0
sec_saved:0.0
total_cost:4977348.730753487
total_count:1223
total_cost before:4977348.730753487
imp_cost:0.0
sec_saved:0.0
total_cost:4977348.730753487
total_count:1224
total_cost before:4977348.730753487
imp_cost:0.0
sec_saved:0.0
total_cost:4977348.730753487
total_count:1225
total_cost before:4977348.730753487
imp_cost:0.0
sec_saved:0.0
total_cost:4977348.730753487
total_count:1226
total_cost before:4

...

total_cost before:5199840.381801013
imp_cost:0.0
sec_saved:0.0
total_cost:5199840.381801013
total_count:18166
total_cost before:5199840.381801013
imp_cost:0.0
sec_saved:0.0
total_cost:5199840.381801013
total_count:18167
total_cost before:5199840.381801013
imp_cost:0.0
sec_saved:0.0
total_cost:5199840.381801013
total_count:18168
total_cost before:5199840.381801013
imp_cost:0.0
sec_saved:0.0
total_cost:5199840.381801013
total_count:18169
total_cost before:5199840.381801013
imp_cost:0.0
sec_saved:0.0
total_cost:5199840.381801013
total_count:18170
total_cost before:5199840.381801013
imp_cost:0.0
sec_saved:0.0
total_cost:5199840.381801013
total_count:18171
total_cost before:5199840.381801013
imp_cost:0.0
sec_saved:0.0
total_cost:5199840.381801013
total_count:18172
total_cost before:5199840.381801013
imp_cost:0.0
sec_saved:0.0
total_cost:5199840.381801013
total_count:18173
total_cost before:5199840.381801013
imp_cost:0.0
sec_saved:0.0
total_cost:5199840.381801013
total_count:18174
total_cost

total_cost before:5199840.381801013
imp_cost:0.0
sec_saved:0.0
total_cost:5199840.381801013
total_count:18448
total_cost before:5199840.381801013
imp_cost:0.0
sec_saved:0.0
total_cost:5199840.381801013
total_count:18449
total_cost before:5199840.381801013
imp_cost:0.0
sec_saved:0.0
total_cost:5199840.381801013
total_count:18450
total_cost before:5199840.381801013
imp_cost:0.0
sec_saved:0.0
total_cost:5199840.381801013
total_count:18451
total_cost before:5199840.381801013
imp_cost:0.0
sec_saved:0.0
total_cost:5199840.381801013
total_count:18452
total_cost before:5199840.381801013
imp_cost:0.0
sec_saved:0.0
total_cost:5199840.381801013
total_count:18453
total_cost before:5199840.381801013
imp_cost:0.0
sec_saved:0.0
total_cost:5199840.381801013
total_count:18454
total_cost before:5199840.381801013
imp_cost:0.0
sec_saved:0.0
total_cost:5199840.381801013
total_count:18455
total_cost before:5199840.381801013
imp_cost:0.0
sec_saved:0.0
total_cost:5199840.381801013
total_count:18456
total_cost

total_cost before:5737695.278366564
imp_cost:5035.920667293758
sec_saved:-0.212408977293068
total_cost:5742731.199033857
total_count:18703
total_cost before:5742731.199033857
imp_cost:4384.371388934612
sec_saved:-0.221828499986728
total_cost:5747115.570422792
total_count:18704
total_cost before:5747115.570422792
imp_cost:4921.774297064193
sec_saved:-0.222815183849956
total_cost:5752037.344719856
total_count:18705
total_cost before:5752037.344719856
imp_cost:5317.234233237056
sec_saved:-0.224274439596466
total_cost:5757354.578953093
total_count:18706
total_cost before:5757354.578953093
imp_cost:5370.944589751222
sec_saved:-0.224501561742241
total_cost:5762725.523542844
total_count:18707
total_cost before:5762725.523542844
imp_cost:2695.4684784313267
sec_saved:-0.227236363236603
total_cost:5765420.992021275
total_count:18708
total_cost before:5765420.992021275
imp_cost:10595.060945106356
sec_saved:-0.076643844370743
total_cost:5776016.052966381
total_count:18709
total_cost before:5776016

total_cost:8714862.693304189
total_count:18933
total_cost before:8714862.693304189
imp_cost:18188.782319621252
sec_saved:-1.533370832968638
total_cost:8733051.47562381
total_count:18934
total_cost before:8733051.47562381
imp_cost:15912.63004657345
sec_saved:-1.546374322276665
total_cost:8748964.105670383
total_count:18935
total_cost before:8748964.105670383
imp_cost:20414.348516741407
sec_saved:-0.517350697958754
total_cost:8769378.454187125
total_count:18936
total_cost before:8769378.454187125
imp_cost:18592.865302948714
sec_saved:-1.567436283301995
total_cost:8787971.319490073
total_count:18937
total_cost before:8787971.319490073
imp_cost:18623.952169170658
sec_saved:-1.570057002661623
total_cost:8806595.271659244
total_count:18938
total_cost before:8806595.271659244
imp_cost:18808.557408139448
sec_saved:-1.585619797579598
total_cost:8825403.829067383
total_count:18939
total_cost before:8825403.829067383
imp_cost:18968.87578534404
sec_saved:-1.599135134630457
total_cost:8844372.70485

5720     None
5787     None
3252     None
2256     None
5718     None
         ... 
6951     None
12429    None
1488     None
12190    None
1997     None
Length: 18989, dtype: object

In [18]:
total_cost

10675987.131343124

In [19]:
total_count

18989